In [91]:
import random
import numpy as np 

def sample_line(n, p):
    
    dir_vec = np.random.randn(3) ;
    line_point = p ;
    parameter = np.transpose(np.random.randn(n)[np.newaxis]) ;
    line_points = line_point + parameter*dir_vec ;
    
      
    return line_points

In [92]:
import numpy as np

#Outputs n sample points from a random plane and m sample points from a line intersecting said random plane.
def plane_line(n, m) :
    sample_p = sample_plane(n)
    
    #Construct a random line through point sample_p[0]
    line_point = sample_p[0] 
    dir_vec = np.random.randn(3) 
    parameter = np.transpose(np.random.random(m)[np.newaxis])
    sample_l = line_point + parameter*dir_vec
    
    
    #The first element is an array of the n sample points from the plane.
    #The second element is an array of the m sample points from the line.
    return [sample_p, sample_l]

In [93]:
import numpy as np 

# Outputs n sample points from a random plane in R^3 as a matrix of the form nx3.
def sample_plane(n):
    
    normal_vec = 2*(np.random.random(3)-1/2)
    point_plane = 2*(np.random.random(3)-1/2) 
    plane_points = [] 
    
    #Points on the plane must satisfy the equation (n1,n2,n3).(x-p1, y-p2, z-p3) = 0, where . operation 
    #represents the dot product and (n1,n2,n3) is the normal vector and (p1,p2,p3) is a point on the plane.
    #We randomly sample for x and y and solve for z using the above equation. 
    for int in range(n) :
        sample_2coord = 2*(np.random.random(2)-1/2) 
        last_coord = sample_2coord - point_plane[0:2] 
        last_coord = -(np.dot(normal_vec[0:2], last_coord)-
                       normal_vec[2]*point_plane[2])/normal_vec[2] 
        sample_2coord = np.append(sample_2coord, last_coord) 
        plane_points = np.append(plane_points,sample_2coord)
   
    plane_points = np.reshape(plane_points, (n,3))
    
         
    return plane_points

In [94]:
import numpy as np


def sample_sphere(n, c):
    """
    Outputs sample points from a random sphere in R^3 centered at c as a matrix 
    of the form nx3
    :param n: No. of points to be sampled
    :param c: Vector - Coordinates for center of sphere [x,y,z]
    :return: Coordinates of the n points on the sphere [x,y,z]
    Note: We use the last construction found in http://mathworld.wolfram.com/SpherePointPicking.html
    """
    vec = 2*(np.random.random((3, n))-1/2)
    vec /= np.linalg.norm(vec, axis=0)
    recenter = np.repeat(c, n)
    recenter = np.reshape(recenter, (3, n))
    vec = vec + recenter
    x=np.transpose(vec)
    
    
    return x

In [95]:
import numpy as np
from numpy import linalg as LA
#import sample_sphere as ss
#from sample_sphere import *

def plane_sphere(n, c, m) :
    """
    Constructs n sample points from a unit sphere centered at c and
    m sample points from a plane which intersects the unit sphere.
    If there are any k points from the m sample points that is contained
    in the unit sphere, we remove them and outputs only the m-k sample points.
    i.e. we do not produce any sample points inside the unit sphere. 
    """
    sphere = sample_sphere(n, c) 
    
    #We construct the plane containing the first 3 sample points 
    #sample_s[0:3] on the sphere. 
    plane_points = sphere[0:3]
    #Finding the normal vector
    vec1 = plane_points[1]-plane_points[0] 
    vec2 = plane_points[2]-plane_points[0] 
    #Normal vector 
    normal_vec = np.cross(vec1, vec2) 
    
    #Points on the plane must satisfy the equation 
    #(n1,n2,n3).(x-p1, y-p2, z-p3) = 0, where . operation 
    #represents the dot product and (n1,n2,n3) is the normal vector and 
    #(p1,p2,p3) is a point on the plane (we call it p0 in the code below).
    #We randomly sample for x and y and solve for z using the above equation. 
    p0=plane_points[0]
    for int in range(m) :
        sample_coord = 2*(np.random.random(2)-1/2) 
        first_coords = sample_coord - p0[0:2]
        last_coord = -(np.dot(normal_vec[0:2], first_coords)-
                       normal_vec[2]*p0[2])/normal_vec[2] 
        sample_coord = np.append(sample_coord, last_coord) 
        plane_points = np.append(plane_points,sample_coord)
    
    plane_points = np.reshape(plane_points, (m+3,3))
    
    #We remove any points on the plane that is inside the unit sphere.
    relevant_points = []
    for i in range(m+3) :
        if LA.norm(plane_points[i]-c)>=1 :
            relevant_points = np.append(relevant_points, plane_points[i])
        
    relevant_points =np.reshape(relevant_points, (-1,3))
    

    
    return [sphere, relevant_points]

In [96]:
import numpy as np
from numpy import linalg as LA
def line_plane_sphere(k, n, c, m) :
    
    ps = plane_sphere(n, c, m) 
    plane = ps[0] 
    sphere = ps[1] 
    #combin_ps = np.reshape(np.append(sphere, plane), (-1,3)) 
    plane_tup = list(tuple(row) for row in plane)
    sphere_tup = list(tuple(row) for row in sphere)
    intersection = np.array([list(item) for item in 
                    set(plane_tup).intersection(sphere_tup)]) 
    if len(intersection) == 0 :
        return ["intersection empty", intersection]
    line = sample_line(k, intersection[0])
    
    #We remove any points on the plane that is inside the unit sphere.
    rel_line_pts = []
    for i in range(k) :
        if LA.norm(line[i]-c)>=1 :
            rel_line_pts = np.append(rel_line_pts, line[i])
        
    rel_line_pts =np.reshape(rel_line_pts, (-1,3))
    
    
    return rel_line_pts, plane, sphere

In [97]:
line,plane,sphere = line_plane_sphere(50, 500, [0,0,0], 500)

In [98]:
# line = tf.convert_to_tensor(line)

sphere_plane=np.concatenate((sphere, plane), axis=0)

In [88]:
plane

<tf.Tensor 'Const_15:0' shape=(500, 3) dtype=float64>

In [78]:
plane_sphere

<tf.Tensor 'concat:0' shape=(794, 3) dtype=float64>

In [1]:
import tensorflow as tf

In [2]:
hello = tf.constant('Hello, TensorFlow!')

In [3]:
sess = tf.Session()

In [4]:
print(sess.run(hello))

b'Hello, TensorFlow!'


In [28]:
import numpy as np

In [14]:
a = tf.constant(np.arange(1, 13, dtype=np.int32),
                shape=[50, 2, 3])

In [15]:
b = tf.constant(np.arange(13, 25, dtype=np.int32),
                shape=[50, 3, 2])

In [16]:
c = tf.matmul(a, b)

In [22]:
c[:2]

<tf.Tensor 'strided_slice_3:0' shape=(2, 2, 2) dtype=int32>

In [17]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const_5:0", shape=(), dtype=float32) Tensor("Const_6:0", shape=(), dtype=float32)


In [31]:
a = tf.constant([1, 2, 3, 4, 5, 6], shape=[2, 3],dtype=tf.float32)

In [32]:
b = tf.constant([7, 8, 9, 10, 11, 12], shape=[3, 2],dtype=tf.float32)

In [129]:
c = tf.matmul(a, b) 

In [130]:
c.get_shape()
tf.Print(c, [c], message="This is a: ")

<tf.Tensor 'Print_1:0' shape=(2, 2) dtype=float32>

In [34]:
s, u, v =tf.svd(c)
# s = svd(a, compute_uv=False)

In [35]:
s

<tf.Tensor 'Svd_2:0' shape=(2,) dtype=float32>

In [18]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [150]:
k=0.95
num=5
rads = [(1/2.)**(i) for i in range(num)]

In [151]:
import numpy as np
import scipy.spatial as spatial
def neighborhood(data, point, rad) :
    coord_set = list(tuple(row) for row in data)
    tree = spatial.cKDTree(coord_set)
    
    return tree.data[tree.query_ball_point(point, rad)]

In [152]:

dims = np.zeros((len(sphere_plane), num))
int_dim = np.zeros(len(sphere_plane))
elem_ind = -1 
count=0

In [153]:
len(sphere_plane)

758

In [ ]:
start = time.clock() 
    a()
    elapsed = time.clock()
    elapsed = elapsed - start
    print "Time spent in (function name) is: ", elapsed

In [213]:

import time
start = time.clock() 
k=0.95
num=5
rads = [(1/2.)**(i) for i in range(num)]
dims = np.zeros((len(sphere_plane), num))
int_dim = np.zeros(len(sphere_plane))
elem_ind = -1 
count=0

for elem in sphere_plane :
    count=count+1
    print(count)
    density = []
    elem_ind += 1
    rad_ind = -1
    for radius in rads :
        Dn = neighborhood(sphere_plane, elem, radius)
        Dn_tf = tf.convert_to_tensor(Dn)
        rad_ind += 1
        if len(Dn) < math.log(k*100*math.log(k*100)):
#                 print("The neighborhood does not have enough points.")
            density.append(0)
            dims[elem_ind, rad_ind] = -1
        else:
            density.append(1)
            Dn_len=len(Dn)
            #print(Dn_len)
            Dn = tf.convert_to_tensor(Dn)
            mean, var = tf.nn.moments(Dn, [1], keep_dims=True)
            Dn0= tf.subtract(Dn, mean)
            scal=(1/(Dn_len-1))**(1/2.)
            #print("this is scalar",scal)
            Dn0 = tf.scalar_mul(scal,Dn0)
            S =tf.svd(Dn0, compute_uv=False)
            S_T=tf.transpose(S,perm=None,name='transpose')
           # print(tf.shape(S))
            S_2 = tf.tensordot(S,S_T,axes=1)
            TV=tf.reduce_sum(S_2)
            CRV=tf.div(S_2, TV)
            k_ten=tf.fill([Dn_len,3], 0.95)
            k_ten = tf.cast(k_ten, tf.float64)
            CRV_bool=tf.greater_equal(CRV, k_ten)
            CRV_bool = tf.cast(CRV_bool, tf.float64)
            CRV_bool=tf.reduce_sum(CRV_bool)
            ult=tf.fill([Dn_len,3], 4)
            ult= tf.cast(ult, tf.float64)
            CRV_bool=tf.subtract(ult,CRV_bool)
            dims[elem_ind, rad_ind] = np.asarray(CRV_bool)
            
elapsed = time.clock()
elapsed = elapsed - start
print ("Time spent in (function name) is: ", elapsed)

1


ValueError: setting an array element with a sequence.

In [116]:
math.log(95*math.log(95))

6.0698558258508974

In [110]:
import math

In [14]:
! pip install numpy scipy
! pip install scikit-learn
! pip install pillow
! pip install h5py


In [15]:


!pip install keras



Exception:
Traceback (most recent call last):
  File "D:\Anaconda3\lib\site-packages\pip\basecommand.py", line 215, in main
    status = self.run(options, args)
  File "D:\Anaconda3\lib\site-packages\pip\commands\install.py", line 335, in run
    wb.build(autobuilding=True)
  File "D:\Anaconda3\lib\site-packages\pip\wheel.py", line 749, in build
    self.requirement_set.prepare_files(self.finder)
  File "D:\Anaconda3\lib\site-packages\pip\req\req_set.py", line 380, in prepare_files
    ignore_dependencies=self.ignore_dependencies))
  File "D:\Anaconda3\lib\site-packages\pip\req\req_set.py", line 554, in _prepare_file
    require_hashes
  File "D:\Anaconda3\lib\site-packages\pip\req\req_install.py", line 278, in populate_link
    self.link = finder.find_requirement(self, upgrade)
  File "D:\Anaconda3\lib\site-packages\pip\index.py", line 465, in find_requirement
    all_candidates = self.find_all_candidates(req.name)
  File "D:\Anaconda3\lib\site-packages\pip\index.py", line 423, in fin

In [19]:
! pip install keras

Exception:
Traceback (most recent call last):
  File "D:\Anaconda3\lib\site-packages\pip\basecommand.py", line 215, in main
    status = self.run(options, args)
  File "D:\Anaconda3\lib\site-packages\pip\commands\install.py", line 335, in run
    wb.build(autobuilding=True)
  File "D:\Anaconda3\lib\site-packages\pip\wheel.py", line 749, in build
    self.requirement_set.prepare_files(self.finder)
  File "D:\Anaconda3\lib\site-packages\pip\req\req_set.py", line 380, in prepare_files
    ignore_dependencies=self.ignore_dependencies))
  File "D:\Anaconda3\lib\site-packages\pip\req\req_set.py", line 554, in _prepare_file
    require_hashes
  File "D:\Anaconda3\lib\site-packages\pip\req\req_install.py", line 278, in populate_link
    self.link = finder.find_requirement(self, upgrade)
  File "D:\Anaconda3\lib\site-packages\pip\index.py", line 465, in find_requirement
    all_candidates = self.find_all_candidates(req.name)
  File "D:\Anaconda3\lib\site-packages\pip\index.py", line 423, in fin

In [20]:
import keras

Using TensorFlow backend.


In [6]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

Using TensorFlow backend.
